In [268]:
#loading packages
import nltk
from nltk.metrics.agreement import AnnotationTask
from nltk.probability import ConditionalFreqDist, FreqDist
import pandas as pd
import os
import numpy as np

In [265]:
#loading data
path='C:\\Users\\Nandakumar\\Documents\\mds\\block 5\\lab1\\LinkedIn_Posting_Markup'
jaskirat_annotated=pd.read_csv(path+'\\data\\jaskirat_annotated.csv',encoding="ISO-8859-1")
jaskirat_annotated=jaskirat_annotated.dropna(how='all')
jaskirat_review=pd.read_csv(path+'\\data\\jaskirat_reviewed.csv',encoding="ISO-8859-1")
jaskirat_review=jaskirat_review.dropna(how='all')
sai_annotated=pd.read_csv(path+'\\data\\Sai_Annotated.csv',encoding="ISO-8859-1")
sai_annotated=sai_annotated.dropna(how='all')
sai_review=pd.read_csv(path+'\\data\\Sai_Reviewed.csv',encoding="ISO-8859-1")
sai_review=sai_review.dropna(how='all')
jaya_annotated=pd.read_csv(path+'\\data\\Jaya_annotation.csv',encoding="ISO-8859-1")
jaya_annotated=jaya_annotated.dropna(how='all')
jaya_review=pd.read_csv(path+'\\data\\jaya_reviewed.csv',encoding="ISO-8859-1")
jaya_review=jaya_review.dropna(how='all')
hari_annotated=pd.read_csv(path+'\\data\\hari_annotated.csv',encoding="ISO-8859-1")
hari_annotated=hari_annotated.dropna(how='all')
hari_review=pd.read_csv(path+'\\data\\hari_reviewed.csv',encoding="ISO-8859-1")
hari_review=hari_review.dropna(how='all')

C:\Users\Nandakumar\AppData\Local\Temp\ipykernel_18440\241254884.py:13: DtypeWarning: Columns (1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  jaya_review=pd.read_csv(path+'\\data\\jaya_reviewed.csv',encoding="ISO-8859-1")


In [237]:
#cleaning data sets to bring it to same shape
sai_annotated.columns = ['Unnamed: 0', 'url', 'title', 'job_description', 'seniority_level',
       'employment_type', 'skills', 'class_name', 'experience']
sai_review.drop(columns=['class_name'],inplace=True)
sai_review.columns=['url', 'title', 'job_description', 'seniority_level', 'employment_type',
       'job_id', 'skills', 'class_name', 'experience']
jaya_review.columns=['Column1', 'url', 'title', 'job_description', 'seniority_level',
       'employment_type', 'skills', 'experience', 'class_name']
hari_annotated.columns=['Unnamed: 0', 'url', 'title', 'job_description', 'seniority_level',
       'employment_type', 'skills', 'class_name', 'experience']

In [238]:
#shape of files that were annotated
print('Total Job description annotated by Jaskirat:',jaskirat_annotated.shape)
print('Total Job description annotated by Jaskirat:',jaskirat_review.shape)
print('Total Job description annotated by sai:',sai_annotated.shape)
print('Total Job description annotated by sai:',sai_review.shape)
print('Total Job description annotated by Jaya:',jaya_annotated.shape)
print('Total Job description annotated by Jaya:',jaya_review.shape)
print('Total Job description annotated by Hari:',hari_annotated.shape)
print('Total Job description annotated by Hari:',hari_review.shape)

Total Job description annotated by Jaskirat: (70, 9)
Total Job description annotated by Jaskirat: (72, 9)
Total Job description annotated by sai: (72, 9)
Total Job description annotated by sai: (70, 9)
Total Job description annotated by Jaya: (70, 9)
Total Job description annotated by Jaya: (66, 9)
Total Job description annotated by Hari: (66, 9)
Total Job description annotated by Hari: (70, 9)


In [258]:
#creating comparison document
final_annotated_data=pd.concat([jaskirat_annotated[['url','title','job_description','skills','experience','class_name']]
           ,sai_annotated[['url','title','job_description','skills','experience','class_name']]
           ,jaya_annotated[['url','title','job_description','skills','experience','class_name']]
          ,hari_annotated[['url','title','job_description','skills','experience','class_name']]],axis=0)
print('Final Annotation document :',final_annotated_data.shape)
final_secondary_annotation_data=pd.concat([sai_review[['url','title','job_description','skills','experience','class_name']]
           ,jaskirat_review[['url','title','job_description','skills','experience','class_name']]
           ,hari_review[['url','title','job_description','skills','experience','class_name']]
          ,jaya_review[['url','title','job_description','skills','experience','class_name']]],axis=0)
print('Final Secondary Annotation document :',final_secondary_annotation_data.shape)

Final Annotation document : (278, 6)
Final Secondary Annotation document : (278, 6)


In [259]:
#Data preprocessing
def cleaning_data(data_frame):
    data_frame['skills']=data_frame.skills.str.lower()
    data_frame['skills']=data_frame.skills.str.strip()
    # data_frame['skills']=data_frame.skills.str.replace(' ', '_')
    data_frame['experience']=data_frame.experience.str.lower()
    data_frame['experience']=data_frame.experience.str.strip()
    data_frame['experience']=data_frame.experience.str.replace(' ', '_')
    data_frame['class_name']=data_frame.class_name.str.lower()
    data_frame['class_name']=data_frame.class_name.str.strip()
    data_frame['class_name']=data_frame.class_name.str.replace(' ', '_')
    return data_frame
final_annotated_data = cleaning_data(final_annotated_data)
final_secondary_annotation_data = cleaning_data(final_secondary_annotation_data)

In [266]:
#annotation task scores
s_score = []
pi_score = []
k_score = []
alpha_score = []
for i in range(final_annotated_data.shape[0]):
    check = []
    check.append(('annotator_1','experience',final_annotated_data.iloc[i,4]))
    check.append(('annotator_2','experience',final_secondary_annotation_data.iloc[i,4]))
    check.append(('annotator_1','class_name',final_annotated_data.iloc[i,5]))
    check.append(('annotator_2','class_name',final_secondary_annotation_data.iloc[i,5]))
    try:
        for skill in final_annotated_data.iloc[i,3].split(","):
            skill = skill.strip()
            skill = skill.replace(' ', '_')
            check.append(('annotator_1','skill',skill))
        for skill in final_secondary_annotation_data.iloc[i,3].split(","):
            skill = skill.strip()
            skill = skill.replace(' ', '_')
            check.append(('annotator_2','skill',skill))
        score = AnnotationTask(check) 
        s_score.append(score.S())
        pi_score.append(score.pi())
    except:
        pass
    k_score.append(score.kappa())
    alpha_score.append(score.alpha())

In [269]:
#Inter-agreement annotation scores
print("S score: ", np.mean(s_score))
print("Pi score: ",np.mean(pi_score))
print("Kappa score: ",np.median(k_score))
print("Alpha score: ",np.mean(alpha_score))

S score:  0.6856323802832042
Pi score:  0.7078020445822651
Kappa score:  0.5714285714285714
Alpha score:  0.20427287589125628
